# Elastic Net Regression

In [50]:
%matplotlib inline
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import pickle
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import cross_val_score,KFold
from sklearn.linear_model import ElasticNet, ElasticNetCV
import warnings


warnings.filterwarnings("ignore")

# environment settings
data_path = 'Data/'

# Deserialize previously saved data from "preprocessing"
with open(data_path+'train_pp.obj', 'rb') as train_pp, \
open(data_path+'test_pp.obj','rb') as test_pp:
    train_df = pickle.load(train_pp)
    test_df = pickle.load(test_pp)
train_df["SalePrice"] = np.log1p(train_df["SalePrice"])

In [51]:
X = train_df.loc[:,'Id':'SaleCondition_Partial']
y = train_df['SalePrice']
print("Shape of training set {}.\nShape of test set {}".format(X.shape,y.shape))

Shape of training set (1460, 303).
Shape of test set (1460,)


In [52]:
ENet = ElasticNetCV(alphas = [1, 0.1, 0.001, 0.0005],
                      cv=KFold(10, shuffle=True, 
                       random_state=1)).fit(X, y)
kf = KFold(5, shuffle=True, random_state=42).get_n_splits(X)
rmse_cv_enet= np.sqrt(-cross_val_score(ENet, X, y, scoring="neg_mean_squared_error", cv = kf))


print("The 10-fold crossvalidation RMSE of ENet is {:.5f} +/- {:.3f} , alpha :{}".format(rmse_cv_enet.mean(),
                                                                                          rmse_cv_enet.std(),
                                                                                          ENet.alpha_))

The 10-fold crossvalidation RMSE of ENet is 0.12564 +/- 0.015 , alpha :0.001


In [53]:
ENet = ElasticNet(alpha=0.001, l1_ratio=.45, random_state=1)
kf = KFold(10, shuffle=True, random_state=42).get_n_splits(X)
rmse_cv_enet= np.sqrt(-cross_val_score(ENet, X, y, scoring="neg_mean_squared_error", cv = kf))
print("The 10-fold crossvalidation RMSE of Elastic Net is {:.5f} +/- {:.3f}".format(rmse_cv_enet.mean(),
                                                                                          rmse_cv_enet.std()))


The 10-fold crossvalidation RMSE of Elastic Net is 0.12269 +/- 0.023


In [54]:
#Prediction on real test set using ENet
ENet = ElasticNet(alpha=0.001,l1_ratio=.45)
ENet.fit(X,y)
pred_results =ENet.predict(test_df)
pred_results = np.expm1(pred_results)
result_df = pd.DataFrame(data={'Id': test_df["Id"].values,
                               'SalePrice': pred_results})
#Create output csv file
result_df.to_csv(data_path+"outputs/enet_001", index=False)